In [305]:
from sklearn.preprocessing import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy
from sklearn import linear_model
from tensorflow.keras.preprocessing.text import Tokenizer

In [420]:
def splitDataset(train, test):
    train = pd.read_csv(train).fillna("na")
    test = pd.read_csv(test).fillna("na")
    x = train.values
    x_Test = test.values
    X_Train = x[:,1:-1]
    X_Train = pd.DataFrame(X_Train)
    Y_Train = x[:,-1]
    Y_Train = pd.DataFrame(Y_Train)
    X_Test = x_Test[:,1:]
    X_Test = pd.DataFrame(X_Test)
    return X_Train,Y_Train,X_Test

In [422]:
def buildVocab(df,testDf):
    d = {}
    cnt = 0
    for i in range(df.shape[1]):
        for j in range(df.shape[0]):
            if type(df[i].iloc[j]) == type("hi"):
                if (df[i].iloc[j]) not in d.keys():
                    d[df[i].iloc[j]] = cnt
                    df[i].iloc[j] = cnt
                    cnt+=1
                else:
                    temp = d[df[i].iloc[j]]
                    df[i].iloc[j] = temp
    for i in range(testDf.shape[1]):
        for j in range(testDf.shape[0]):
            if type(testDf[i].iloc[j]) == type("hi"):
                if (testDf[i].iloc[j]) not in d.keys():
                    d[testDf[i].iloc[j]] = cnt
                    testDf[i].iloc[j] = cnt
                    cnt+=1
                else:
                    temp = d[testDf[i].iloc[j]]
                    testDf[i].iloc[j] = temp

    print(d)
    return df, testDf

In [423]:
train = "./LR/train.csv"
test = "./LR/test.csv"
X_Train, Y_Train, X_Test = splitDataset(train,test)
# X_Train, Y_Train = pDat(train, test)
# X_Train = plWork(X_Train)
X_Train.head(10)
# print(X_Test)

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,60,RL,65,8450,Pave,na,Reg,Lvl,AllPub,Inside,...,0,0,na,na,na,0,2,2008,WD,Normal
1,20,RL,80,9600,Pave,na,Reg,Lvl,AllPub,FR2,...,0,0,na,na,na,0,5,2007,WD,Normal
2,60,RL,68,11250,Pave,na,IR1,Lvl,AllPub,Inside,...,0,0,na,na,na,0,9,2008,WD,Normal
3,70,RL,60,9550,Pave,na,IR1,Lvl,AllPub,Corner,...,0,0,na,na,na,0,2,2006,WD,Abnorml
4,60,RL,84,14260,Pave,na,IR1,Lvl,AllPub,FR2,...,0,0,na,na,na,0,12,2008,WD,Normal
5,50,RL,85,14115,Pave,na,IR1,Lvl,AllPub,Inside,...,0,0,na,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75,10084,Pave,na,Reg,Lvl,AllPub,Inside,...,0,0,na,na,na,0,8,2007,WD,Normal
7,60,RL,na,10382,Pave,na,IR1,Lvl,AllPub,Corner,...,0,0,na,na,Shed,350,11,2009,WD,Normal
8,50,RM,51,6120,Pave,na,Reg,Lvl,AllPub,Inside,...,0,0,na,na,na,0,4,2008,WD,Abnorml
9,190,RL,50,7420,Pave,na,Reg,Lvl,AllPub,Corner,...,0,0,na,na,na,0,1,2008,WD,Normal


In [424]:
X_Train,X_Test = buildVocab(X_Train,X_Test)
X_Train.head(20)

{'RL': 0, 'RM': 1, 'C (all)': 2, 'FV': 3, 'RH': 4, 'na': 5, 'Pave': 6, 'Grvl': 7, 'Reg': 8, 'IR1': 9, 'IR2': 10, 'IR3': 11, 'Lvl': 12, 'Bnk': 13, 'Low': 14, 'HLS': 15, 'AllPub': 16, 'NoSeWa': 17, 'Inside': 18, 'FR2': 19, 'Corner': 20, 'CulDSac': 21, 'FR3': 22, 'Gtl': 23, 'Mod': 24, 'Sev': 25, 'CollgCr': 26, 'Veenker': 27, 'Crawfor': 28, 'NoRidge': 29, 'Mitchel': 30, 'Somerst': 31, 'NWAmes': 32, 'OldTown': 33, 'BrkSide': 34, 'Sawyer': 35, 'NridgHt': 36, 'NAmes': 37, 'SawyerW': 38, 'IDOTRR': 39, 'MeadowV': 40, 'Edwards': 41, 'Timber': 42, 'Gilbert': 43, 'StoneBr': 44, 'ClearCr': 45, 'NPkVill': 46, 'Blmngtn': 47, 'BrDale': 48, 'SWISU': 49, 'Blueste': 50, 'Norm': 51, 'Feedr': 52, 'PosN': 53, 'Artery': 54, 'RRAe': 55, 'RRNn': 56, 'RRAn': 57, 'PosA': 58, 'RRNe': 59, '1Fam': 60, '2fmCon': 61, 'Duplex': 62, 'TwnhsE': 63, 'Twnhs': 64, '2Story': 65, '1Story': 66, '1.5Fin': 67, '1.5Unf': 68, 'SFoyer': 69, 'SLvl': 70, '2.5Unf': 71, '2.5Fin': 72, 'Gable': 73, 'Hip': 74, 'Gambrel': 75, 'Mansard': 76

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,60,0,65,8450,6,5,8,12,16,18,...,0,0,5,5,5,0,2,2008,160,169
1,20,0,80,9600,6,5,8,12,16,19,...,0,0,5,5,5,0,5,2007,160,169
2,60,0,68,11250,6,5,9,12,16,18,...,0,0,5,5,5,0,9,2008,160,169
3,70,0,60,9550,6,5,9,12,16,20,...,0,0,5,5,5,0,2,2006,160,170
4,60,0,84,14260,6,5,9,12,16,19,...,0,0,5,5,5,0,12,2008,160,169
5,50,0,85,14115,6,5,9,12,16,18,...,0,0,5,153,78,700,10,2009,160,169
6,20,0,75,10084,6,5,8,12,16,18,...,0,0,5,5,5,0,8,2007,160,169
7,60,0,5,10382,6,5,9,12,16,20,...,0,0,5,5,78,350,11,2009,160,169
8,50,1,51,6120,6,5,8,12,16,18,...,0,0,5,5,5,0,4,2008,160,170
9,190,0,50,7420,6,5,8,12,16,20,...,0,0,5,5,5,0,1,2008,160,169


In [425]:
from sklearn.metrics import mean_squared_error
regr = linear_model.LinearRegression()
regr.fit(X_Train,Y_Train)
trainPred = regr.predict(X_Train)
mean_squared_error(trainPred,Y_Train.values)

937338126.0456618

In [431]:
import sys
numpy.set_printoptions(threshold=sys.maxsize)
predicted = regr.predict(X_Test)
toStr = str(predicted)
f = open("pred.txt", "w")
f.write(toStr)
f.close()
print(predicted)
print(predicted[:50])

[[ 118959.22535391]
 [ 152792.92504709]
 [ 175109.2221149 ]
 [ 202256.70290246]
 [ 193002.78981679]
 [ 180841.67782206]
 [ 189932.53804567]
 [ 167621.5437965 ]
 [ 194603.07108595]
 [ 116300.65506433]
 [ 198756.47402514]
 [ 107765.64583226]
 [  83583.06020726]
 [ 148054.4441118 ]
 [ 105590.95257012]
 [ 339231.55499555]
 [ 243175.155936  ]
 [ 306810.06380179]
 [ 297869.05700464]
 [ 447794.47065516]
 [ 296307.66979427]
 [ 202671.27510547]
 [ 171634.63710609]
 [ 157399.46055276]
 [ 186410.50275089]
 [ 201166.91998741]
 [ 316428.34163115]
 [ 312305.32188252]
 [ 189253.74661191]
 [ 249327.89379059]
 [ 196459.23683942]
 [  99851.45589341]
 [ 211177.7124248 ]
 [ 285482.81044903]
 [ 273913.80501011]
 [ 234354.39125545]
 [ 170541.67761731]
 [ 156601.1832191 ]
 [ 156781.28000433]
 [ 146540.11188729]
 [ 193856.60154633]
 [ 147657.21055975]
 [ 268706.65921795]
 [ 242451.41107226]
 [ 219563.23208885]
 [ 211964.61831505]
 [ 249713.56320859]
 [ 213038.70690257]
 [ 160681.00631747]
 [ 146091.80231647]
